In [1]:
from bs4 import BeautifulSoup 
from urllib.request import urlopen

import pandas as pd
import re
# 웹크롤링한 데이터 읽어오기
# df = pd.read_csv('data/exam-WebParsing.csv')       
df = pd.read_csv('data/chicagomag_info.csv', index_col='Rank') 
df.head()

,Unnamed: 0,Cafe,Menu,URL
Rank,,,,
1,0,Old Oak Tap,BLT,http://www.chicagomag.com/Chicago-Magazine/Nov...
2,1,Au Cheval,Fried Bologna,http://www.chicagomag.com/Chicago-Magazine/Nov...
3,2,Xoco,Woodland Mushroom,http://www.chicagomag.com/Chicago-Magazine/Nov...
4,3,Al’s Deli,Roast Beef,http://www.chicagomag.com/Chicago-Magazine/Nov...
5,4,Publican Quality Meats,PB&L,http://www.chicagomag.com/Chicago-Magazine/Nov...


In [2]:
# index_col='Unnamed: 0'') 삭제
del df['Unnamed: 0']
df.head()

,Cafe,Menu,URL
Rank,,,
1,Old Oak Tap,BLT,http://www.chicagomag.com/Chicago-Magazine/Nov...
2,Au Cheval,Fried Bologna,http://www.chicagomag.com/Chicago-Magazine/Nov...
3,Xoco,Woodland Mushroom,http://www.chicagomag.com/Chicago-Magazine/Nov...
4,Al’s Deli,Roast Beef,http://www.chicagomag.com/Chicago-Magazine/Nov...
5,Publican Quality Meats,PB&L,http://www.chicagomag.com/Chicago-Magazine/Nov...


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 50 entries, 1 to 50
Data columns (total 3 columns):
Cafe    50 non-null object
Menu    50 non-null object
URL     50 non-null object
dtypes: object(3)
memory usage: 1000.0+ bytes


In [4]:
df['Menu'][1]
'BLT'
df['URL'][1]

'http://www.chicagomag.com/Chicago-Magazine/November-2012/Best-Sandwiches-in-Chicago-Old-Oak-Tap-BLT/'

In [5]:
from images import bigpycraft_bda as bpc
bpc.Figure(bpc.BDA_WC_12, 800)

ModuleNotFoundError: No module named 'images'

In [6]:
html = urlopen(df['URL'][1])
soup = BeautifulSoup(html, "lxml")
soup.title
# type(soup)

<title>
  1. Old Oak Tap BLT |
  Chicago magazine
      |  November 2012
    </title>

In [7]:
soup.find('p', 'addy')

<p class="addy">
<em>$10. 2109 W. Chicago Ave., 773-772-0406, <a href="http://www.theoldoaktap.com/">theoldoaktap.com</a></em></p>

In [8]:
taginfo = soup.find('p', 'addy').get_text()
taginfo

'\n$10. 2109 W. Chicago Ave., 773-772-0406, theoldoaktap.com'

In [9]:
price = re.search('\$\d+\.(\d+)?', taginfo).group()
price

'$10.'

In [10]:
address = '일단보류'    # Street, City
phone = re.search('\d{3}[-]\d{3}[-]\d{4}', taginfo)
if phone is not None:
    phone = phone.group()
phone
'773-772-0406'

'773-772-0406'

In [11]:
infolist = taginfo.split(', ')
if infolist[-1].endswith('.com') or infolist[-1].endswith('.net'):
    homepage = infolist[-1]
else:
    homepage = 'No Homepage'
homepage
'theoldoaktap.com'

'theoldoaktap.com'

In [13]:

'theoldoaktap.com'
start = len(price)
if phone is None:
    phone = 'No Contact'
    end = taginfo.find(homepage)
else:
    end = taginfo.find(phone)
    
address = taginfo[start:end]
address
' 2109 W. Chicago Ave., '
# 상위 TOP3 페이지에서 가격과 주소정보만 읽어오기
Price    = []
Address  = []
Phone    = []
Homepage = []

for n in [1,2,3,4,5]:   
    html = urlopen(df['URL'][n])
    soup = BeautifulSoup(html, 'lxml')
    
    taginfo = soup.find('p', 'addy').get_text()
    taginfo = taginfo[taginfo.find('$'):]
    
    price = re.search('\$\d+\.(\d+)?', taginfo).group()
    
    address = '일단보류'    # Street, City
    
    phone = re.search('\d{3}[-]\d{3}[-]\d{4}', taginfo)
    if phone is not None:
        phone = phone.group()    
        
    infolist = taginfo.split(', ')
    if infolist[-1].endswith('.com') or infolist[-1].endswith('.net'):
        homepage = infolist[-1]
    else:
        homepage = 'No Homepage'

    start = len(price)
    if phone is None:
        phone = 'No Contact'
        end = taginfo.find(homepage)
    else:
        end = taginfo.find(phone)

    address = taginfo[start:end] 

    Price.append(price)
    Address.append(address)
    Phone.append(phone)
    Homepage.append(homepage)
    
    print(n)

1
2
3
4
5


In [15]:
Price

['$10.', '$9.', '$9.50', '$9.40', '$10.']

In [17]:
Phone

['773-772-0406',
 '312-929-4580',
 '312-334-3688',
 '847-475-9400',
 '312-445-8977']

In [16]:
Address

[' 2109 W. Chicago Ave., ',
 ' 800 W. Randolph St., ',
 '. 445 N. Clark St., ',
 '. 914 Noyes St., Evanston, ',
 ' 825 W. Fulton Mkt., ']

In [18]:
Homepage

['theoldoaktap.com',
 'aucheval.tumblr.com',
 'rickbayless.com',
 'alsdeli.net',
 'publicanqualitymeats.com']

In [19]:
price = Price[0]
if price.endswith('.'):
    price = price[:-1]
price

'$10'

In [20]:
address = Address[2]
address = address[:-2]
if address.startswith('.'):
    address = address[1:]
address = address.strip()
address

'445 N. Clark St.'

In [21]:
Price    = []
Address  = []
Phone    = []
Homepage = []

for n in [1,2,3,4,5]:   
    html = urlopen(df['URL'][n])
    soup = BeautifulSoup(html, 'lxml')
    
    taginfo = soup.find('p', 'addy').get_text()
    taginfo = taginfo[taginfo.find('$'):]
    
    price = re.search('\$\d+\.(\d+)?', taginfo).group()
    # price update
    if price.endswith('.'):
        price = price[:-1]
        
    address = '일단보류'    # Street, City
    
    phone = re.search('\d{3}[-]\d{3}[-]\d{4}', taginfo)
    if phone is not None:
        phone = phone.group()    
        
    infolist = taginfo.split(', ')
    if infolist[-1].endswith('.com') or infolist[-1].endswith('.net'):
        homepage = infolist[-1]
    else:
        homepage = 'No Homepage'

    start = len(price)
    if phone is None:
        phone = 'No Contact'
        end = taginfo.find(homepage)
    else:
        end = taginfo.find(phone)

    address = taginfo[start:end] 
    # address update
    address = address[:-2]
    if address.startswith('.'):
        address = address[1:]
    address = address.strip()
    
    Price.append(price)
    Address.append(address)
    Phone.append(phone)
    Homepage.append(homepage)
    
    print(n)

1
2
3
4
5


In [22]:
Price

['$10', '$9', '$9.50', '$9.40', '$10']